In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PATH_TO_POSTERIORS = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/results/posterior_samples/'
PATH_TO_PLOTS = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/plots/samples_plots/'
PATH_TO_INFO = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/info/'
PATH_TO_INJECTIONS = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/injections/'

In [3]:
waveform = 'IMRPhenomXPHM'
estimator = 'median'

In [4]:
events = np.loadtxt(PATH_TO_INFO + 'events_wf_%s_priors_%s.txt' %(estimator, waveform), dtype=str)
with open(PATH_TO_INFO + 'detectors_dictionary.pkl', 'rb') as f:
    detectors = pickle.load(f)

In [5]:
params = ['chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 
            'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl']

In [6]:
my_colors = [(1.0, 0.4980392156862745, 0.054901960784313725), #orange
            (0.12156862745098039, 0.4666666666666667, 0.7058823529411765)] #blue

In [7]:
param_label = {'chirp_mass': r'$\mathcal{M}_c$', 'mass_ratio': r'$q$', 'luminosity_distance': r'$d_L$',
                'dec': r'$DEC$', 'ra': r'$RA$', 'theta_jn': r'$\theta_{JN}$', 'psi': r'$\Psi$',
                'phase': r'$\phi$', 'geocent_time': r'$t_c$', 'a_1': r'$a_1$', 'a_2': r'$a_2$',
                'tilt_1': r'$tilt_1$', 'tilt_2': r'$tilt_2$', 'phi_12': r'$phi_{12}$',
                'phi_jl': r'$phi_{JL}$'}

In [8]:
param_unit = {'chirp_mass': r'[$M_{\odot}$]', 'mass_ratio': r'', 'luminosity_distance': r'[Mpc]',
                'dec': r'[rad]', 'ra': r'[rad]', 'theta_jn': r'[rad]', 'psi': r'[rad]',
                'phase': r'[rad]', 'geocent_time': r'[s]', 'a_1': r'', 'a_2': r'',
                'tilt_1': r'[rad]', 'tilt_2': r'[rad]', 'phi_12': r'[rad]',
                'phi_jl': r'[rad]'}

In [9]:
for event in events:

    injections = pd.read_hdf(PATH_TO_INJECTIONS + event + '/%s_%s_%s.hdf5' %(event, waveform, estimator))
    lvk_samples = pd.read_hdf(PATH_TO_POSTERIORS + 'lvk_samples/lvk_samples_%s.hdf5' %event)
    fisher_samples = pd.read_hdf(PATH_TO_POSTERIORS + 'fisher_samples/fisher_samples_%s_%s.hdf5' %(estimator, event))
    fisher_plus_priors_samples = pd.read_hdf(PATH_TO_POSTERIORS + 'fisher_plus_priors_samples/fisher_plus_priors_samples_%s_%s.hdf5' %(estimator, event))

    fig, axs = plt.subplots(5, 3, figsize=(15, 10))

    for i, param in enumerate(params):
        ax = axs[i//3, i%3]
        ax = sns.histplot(lvk_samples[param], element='step', ax=ax, label='LVK', alpha = 0.5, color = 'black')
        ax.axvline(injections[param].values[0], color='red', label = 'injection', linewidth=1.5)
        ax = sns.histplot(fisher_samples[param], element='step', ax=ax, label='Fisher', alpha = 0.25, color = my_colors[0])
        ax = sns.histplot(fisher_plus_priors_samples[param], element='step', ax=ax, label='Fisher + Priors', alpha=0.5, color = my_colors[1])
        
        ax.set_ylabel('Count', fontsize=12)
        ax.set_xlabel('%s %s' %(param_label[param], param_unit[param]), fontsize=12)
        ax.set_yscale('log')
        ax.tick_params(axis='both', which='major', labelsize=12)
        ax.tick_params(axis='both', which='minor', labelsize=12)
        ax.legend(fontsize=9)

    plt.suptitle('%s' %event, fontsize=15)
    plt.tight_layout()
    plt.savefig(PATH_TO_PLOTS + 'samples_comparison_%s_%s.png' %(estimator, event), dpi=300)
    plt.close()